# Intro to Hadoop

In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from nose.tools import assert_equal, assert_true, assert_is_instance
from numpy.testing import assert_array_almost_equal, assert_almost_equal

import warnings
warnings.filterwarnings('ignore')

In [9]:
!$HADOOP_HOME/sbin/stop-dfs.sh
!$HADOOP_HOME/sbin/stop-yarn.sh
!rm -rf /tmp/*
!echo "Y" | $HADOOP_HOME/bin/hdfs namenode -format 2> /dev/null
!$HADOOP_HOME/etc/hadoop/hadoop-env.sh
!$HADOOP_HOME/sbin/start-dfs.sh
!$HADOOP_HOME/sbin/start-yarn.sh
!$HADOOP_HOME/bin/hdfs dfsadmin -safemode leave
!$HADOOP_HOME/bin/hdfs dfs -mkdir -p /user/$NB_USER

17/08/19 11:16:21 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Stopping namenodes on [localhost]
localhost: stopping namenode
localhost: stopping datanode
Stopping secondary namenodes [0.0.0.0]
0.0.0.0: stopping secondarynamenode
17/08/19 11:16:41 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
stopping yarn daemons
stopping resourcemanager
localhost: stopping nodemanager
localhost: nodemanager did not stop gracefully after 5 seconds: killing with kill -9
no proxyserver to stop
Formatting using clusterid: CID-f58395ad-9f2a-432c-ba91-0055985eb362
/bin/sh: /usr/local/Cellar/hadoop/2.8.0/etc/hadoop/hadoop-env.sh: No such file or directory
17/08/19 11:16:56 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Starting namenodes on [localhost]
localho

## Part 1: Exploring files and the system

In [8]:
!$HADOOP_HOME/bin/hdfs dfs -ls /

17/08/18 19:48:20 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 1 items
drwxr-xr-x   - Constance supergroup          0 2017-08-18 19:27 /user


## Part 2 Free space:

In [9]:
!$HADOOP_HOME/bin/hdfs dfs -df -h

17/08/18 19:49:39 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Filesystem                Size  Used  Available  Use%
hdfs://localhost:9000  111.9 G   8 K     25.5 G    0%


## Part 3: Version

In [13]:
!hadoop version

Hadoop 2.8.0
Subversion https://git-wip-us.apache.org/repos/asf/hadoop.git -r 91f2b7a13d1e97be65db92ddabc627cc29ac0009
Compiled by jdu on 2017-03-17T04:12Z
Compiled with protoc 2.5.0
From source with checksum 60125541c2b3e266cbf3becc5bda666
This command was run using /usr/local/Cellar/hadoop/2.8.0/libexec/share/hadoop/common/hadoop-common-2.8.0.jar


### New directory for Hadoop

In [14]:
%%bash
#!/usr/bin/env bash

DIR=$HOME/hadoop_assign

# Delete if exists
if [ -d "$DIR" ]; then
    rm -rf "$DIR"
fi

# Now make the directory
mkdir "$DIR"

ls -la $DIR

total 0
drwxr-xr-x    2 Constance  staff    68 Aug 18 20:02 .
drwxr-xr-x+ 141 Constance  staff  4794 Aug 18 20:02 ..


## Part 4: Copying a book into a directory

In [15]:
!$HADOOP_HOME/bin/hdfs dfs -mkdir -p grail/in
!$HADOOP_HOME/bin/hdfs dfs -put $HOME/hadoop_assign/grail.txt grail/in/grail.txt

17/08/18 20:25:53 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
17/08/18 20:25:55 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Part 5: Making a new directory and copying a book in Hadoop

In [16]:
had_grail = !$HADOOP_HOME/bin/hdfs dfs -count -h grail/in/*
had_grail = had_grail[0].split()
assert_is_instance(had_grail, list)
assert_true(all(isinstance(w, str) for w in had_grail))
assert_true(had_grail, ['0', '1', '63.5', 'K', 'grail/in/grail.txt'])

### Clean up

In [11]:
!rm grail.txt
!$HADOOP_HOME/bin/hdfs dfs -rm -r -f grail
!rm -rf $HOME/hadoop_assign
!$HADOOP_HOME/sbin/stop-dfs.sh
!$HADOOP_HOME/sbin/stop-yarn.sh

rm: grail.txt: No such file or directory
17/08/19 11:18:18 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
17/08/19 11:18:20 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Stopping namenodes on [localhost]
localhost: stopping namenode
localhost: stopping datanode
Stopping secondary namenodes [0.0.0.0]
0.0.0.0: stopping secondarynamenode
17/08/19 11:18:40 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
stopping yarn daemons
stopping resourcemanager
localhost: stopping nodemanager
localhost: nodemanager did not stop gracefully after 5 seconds: killing with kill -9
no proxyserver to stop


# Apache Pig

In [1]:
from nose.tools import assert_equal, assert_almost_equal

## Raw Data Preview

In [2]:
!head -5 $HOME/book-crossing/BX-Book-Ratings.csv

In [3]:
!head -5 $HOME/book-crossing/BX-Books.csv

## Data Preprocessing

In [4]:
%%bash

sed 's/"//g' $HOME/book-crossing/BX-Book-Ratings.csv | sed '1d' > ratings.csv
sed 's/"//g' $HOME/book-crossing/BX-Books.csv | cut -d';' -f -4 | sed '1d' > books.csv

echo
echo '***** Ratings File *****'
head ratings.csv

echo
echo '***** Books File *****'
head books.csv


***** Ratings File *****
276725;034545104X;0
276726;0155061224;5
276727;0446520802;0
276729;052165615X;3
276729;0521795028;6
276733;2080674722;0
276736;3257224281;8
276737;0600570967;6
276744;038550120X;7
276745;342310538;10

***** Books File *****
0195153448;Classical Mythology;Mark P. O. Morford;2002
0002005018;Clara Callan;Richard Bruce Wright;2001
0060973129;Decision in Normandy;Carlo D'Este;1991
0374157065;Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It;Gina Bari Kolata;1999
0393045218;The Mummies of Urumchi;E. J. W. Barber;1999
0399135782;The Kitchen God's Wife;Amy Tan;1991
0425176428;What If?: The World's Foremost Military Historians Imagine What Might Have Been;Robert Cowley;2000
0671870432;PLEADING GUILTY;Scott Turow;1993
0679425608;Under the Black Flag: The Romance and the Reality of Life Among the Pirates;David Cordingly;1996
074322678X;Where You'll Find Me: And Other Stories;Ann Beattie;2002


sed: RE error: illegal byte sequence
sed: RE error: illegal byte sequence


## Pig Latin: Average

In [5]:
%%writefile average.pig

ratings = LOAD 'ratings.csv' USING PigStorage(';') AS (UserID:int, ISBN:chararray, rating:int) ;
books = LOAD 'books.csv' USING PigStorage(';') AS (ISBN:chararray, title:chararray, author:chararray, year:int); 
groups = GROUP ratings BY ISBN ;
avg = FOREACH groups GENERATE group AS GISBN, AVG (ratings.rating) ;
results = JOIN avg by GISBN, books by ISBN ;
ordered_results = ORDER results BY title;
Top_Results = LIMIT ordered_results 10 ;
DUMP Top_Results;

Writing average.pig


In [6]:
average_ratings = !pig -x local -f average.pig 2> pig_stderr.log
print('\n'.join(average_ratings))

(1558746218,0.0,1558746218,A Second Chicken Soup for the Woman's Soul (Chicken Soup for the Soul Series),Jack Canfield,1998)
(0345402871,2.0,0345402871,Airframe,Michael Crichton,1997)
(0452264464,4.3076923076923075,0452264464,Beloved (Plume Contemporary Fiction),Toni Morrison,1994)
(0425163091,0.0,0425163091,Chocolate Jesus,Stephan Jaramillo,1998)
(0002005018,4.333333333333333,0002005018,Clara Callan,Richard Bruce Wright,2001)
(0195153448,0.0,0195153448,Classical Mythology,Mark P. O. Morford,2002)
(0060973129,4.0,0060973129,Decision in Normandy,Carlo D'Este,1991)
(0689821166,6.0,0689821166,Flood : Mississippi 1927,Kathleen Duey,1998)
(0374157065,0.0,0374157065,Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It,Gina Bari Kolata,1999)
(1881320189,7.0,1881320189,Goodbye to the Buttermilk Sky,Julia Oliver,1994)


## Cleanup

In [7]:
%%bash
# Remove pig log files
rm -f pig*.log

# Remove our pig scripts
rm -f *.pig

# Remove csv files
rm books.csv ratings.csv